In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [3]:
# 넘파이 데이터 불러오기
X_train, X_test, Y_train, Y_test = np.load(
    '/content/drive/MyDrive/Colab Notebooks/datas/books_data_max_970_size_131128.npy',
    allow_pickle=True)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(49524, 970)
(8740, 970)
(49524, 8)
(8740, 8)


In [10]:
model = Sequential() 
model.add(Embedding(131128, 200, input_length=970))
# Embedding : 벡터라이징을 해주는 함수, input_length : 문장은 길이만 있기에 length
model.add(Conv1D(32, kernel_size=5,
                 padding='same', activation='relu')) # 문장의 관계 Conv1D
model.add(MaxPool1D(pool_size=1))
# 순서가 있는 데이터, LSTM 사용
model.add(LSTM(128, activation='tanh',
               return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(64, activation='tanh',
               return_sequences = True))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax')) # 다중분류기 softmax를 사용하여 확률값으로
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 970, 200)          26225600  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 970, 32)           32032     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 970, 32)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 970, 128)          82432     
_________________________________________________________________
dropout_6 (Dropout)          (None, 970, 128)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 970, 64)           49408     
_________________________________________________________________
dropout_7 (Dropout)          (None, 970, 64)          

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
fit_hist = model.fit(X_train, Y_train, batch_size=256,
                     epochs=5, validation_data=(X_test, Y_test))

Epoch 1/5
194/194 [==============================] - 100s 503ms/step - loss: 0.9043 - accuracy: 0.6681 - val_loss: 0.4163 - val_accuracy: 0.8696
Epoch 2/5
194/194 [==============================] - 97s 501ms/step - loss: 0.2474 - accuracy: 0.9248 - val_loss: 0.3661 - val_accuracy: 0.8843
Epoch 3/5
194/194 [==============================] - 98s 503ms/step - loss: 0.0914 - accuracy: 0.9736 - val_loss: 0.4235 - val_accuracy: 0.8838
Epoch 4/5
194/194 [==============================] - 97s 502ms/step - loss: 0.0413 - accuracy: 0.9884 - val_loss: 0.5169 - val_accuracy: 0.8812
Epoch 5/5
194/194 [==============================] - 98s 503ms/step - loss: 0.0236 - accuracy: 0.9936 - val_loss: 0.6113 - val_accuracy: 0.8768


In [12]:
score = model.evaluate(X_test, Y_test)
print(score[1])

274/274 [==============================] - 10s 37ms/step - loss: 0.6113 - accuracy: 0.8768
0.876773476600647


In [ ]:
model.save('/content/models/news_classification_{}.h5'.format(score[1]))